# Align and crop a face in an image using dlib

In [20]:
import cv2
import dlib
import numpy as np
import imutils
from imutils.face_utils import shape_to_np

In [21]:
import dlib
print(dlib.__version__)


19.24.0


In [23]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [ ]:
def align_face(image, landmarks, desired_left_eye=(0.35, 0.35), output_size=(224, 224)):
    # Extract the left and right eye coordinates
    left_eye_pts = landmarks[36:42]
    right_eye_pts = landmarks[42:48]

    if left_eye_pts.shape[0] == 0 or right_eye_pts.shape[0] == 0:
        raise ValueError("Eye landmarks not detected properly.")

    left_eye_center = left_eye_pts.mean(axis=0)
    right_eye_center = right_eye_pts.mean(axis=0)

    # Compute the angle between the eyes
    dY = right_eye_center[1] - left_eye_center[1]
    dX = right_eye_center[0] - left_eye_center[0]
    angle = np.degrees(np.arctan2(dY, dX))

    # Desired distance between eyes
    desired_dist = (1.0 - 2 * desired_left_eye[0]) * output_size[0]
    actual_dist = np.linalg.norm(right_eye_center - left_eye_center)
    if actual_dist == 0:
        raise ValueError("Eye distance is zero, cannot align.")

    scale = desired_dist / actual_dist

    # Compute center between eyes
    eyes_center = (
        int((left_eye_center[0] + right_eye_center[0]) / 2),
        int((left_eye_center[1] + right_eye_center[1]) / 2)
    )

    # Get rotation matrix
    M = cv2.getRotationMatrix2D(eyes_center, angle, scale)

    # Adjust translation
    tX = output_size[0] * 0.5
    tY = output_size[1] * desired_left_eye[1]
    M[0, 2] += (tX - eyes_center[0])
    M[1, 2] += (tY - eyes_center[1])

    # Apply affine transformation
    aligned = cv2.warpAffine(image, M, output_size, flags=cv2.INTER_CUBIC)

    return aligned

def detect_align_crop(image_path, save_path="aligned_face.jpg", visualize_landmarks=False):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image {image_path}")
        return

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)

    if len(rects) == 0:
        print("No face detected.")
        return

    rect = rects[0]
    shape = predictor(gray, rect)
    landmarks = shape_to_np(shape)

    print(f"Detected {len(landmarks)} landmarks")
    if landmarks.shape[0] < 68:
        print("Warning: Less than 68 landmarks found.")
        #return

    if visualize_landmarks:
        for (x, y) in landmarks:
            cv2.circle(image, (x, y), 6, (0, 255, 0), -1) # 6 is radius of the dots (just for vizualization)
        cv2.imshow("Landmarks", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    try:
        aligned_face = align_face(image, landmarks)
        cv2.imwrite(save_path, aligned_face)
        print(f"Aligned face saved to {save_path}")
    except Exception as e:
        print(f"Alignment failed: {e}")

In [29]:
# Test:
detect_align_crop("test_face.jpg")

# Test vizualization:
detect_align_crop("test_face.jpg", save_path="aligned_face_viz.jpg", visualize_landmarks=True)


Detected 68 landmarks
Aligned face saved to aligned_face.jpg
Detected 68 landmarks
Aligned face saved to aligned_face_viz.jpg
